In [1]:
## download model to the current directory. Ignore if download manually 
import os, urllib
import tarfile
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)
# def get_model(prefix, epoch):
#     download(prefix+'-symbol.json')
#     download(prefix+'-%04d.params' % (epoch,))
download('http://data.mxnet.io/models/imagenet/inception-v3.tar.gz')

In [17]:
tar = tarfile.open("inception-v3.tar.gz")
tar.extractall()
tar.close()

In [2]:
import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import os
import cv2
import xgboost as xgb
import mxnet as mx
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
def get_extractor():
    model = mx.model.FeedForward.load('model/Inception-7', 0, ctx=mx.gpu(), numpy_batch_size=1)
    fea_symbol = model.symbol.get_internals()["flatten_output"]
    feature_extractor = mx.model.FeedForward(ctx=mx.gpu(), symbol=fea_symbol, numpy_batch_size=16,
                                             arg_params=model.arg_params, aux_params=model.aux_params,
                                             allow_extra_params=True)

    return feature_extractor 

MIN_BOUND = -1500.0
MAX_BOUND = 600.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

PIXEL_MEAN = 0.25

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

def next_batch(sample_image):
    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (299, 299))
            tmp.append(img)

        tmp = np.array(tmp)
        batch.append(np.array(tmp))
    batch = np.array(batch)
    return batch

# def calc_features():
#     net = get_extractor()
#     for folder in glob.glob(path):
#         batch = next_batch(image)
#         feats = net.predict(batch)
#         np.save(os.path.join(BASE_PATH, folder), feats)
#         print folder

# INPUT_FOLDER = '/media/zzz/OS/stage1/'
# patients = os.listdir(INPUT_FOLDER)
# patients.sort()

In [ ]:
file_path = '/media/ai-master/my_passport/joyce/Pre_processed'
save_path = '/media/ai-master/my_passport/joyce/inception'
patients = os.listdir(file_path)
net = get_extractor()

for i in range(len(patients)):
    image = np.load(file_path+'/'+patients[0])
    image = normalize(image)
    image = zero_center(image)
    batch = next_batch(image)
    feats = net.predict(batch)
    np.save(os.path.join(save_path, patients[i]), feats)

/home/ai-master/mxnet/python/mxnet/model.py:870: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
  **kwargs)
/home/ai-master/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
